This notebook gives some simple examples for:
+ Comparing GeneSets to eachother via some machine-learning model score.
+ Ranking genes within a geneset based on some machine-learning model.

The iterations / generations trained in this notebook are low so that the notebook executes in a reasonable amount of time.

***Setting up the notebook***

In [ ]:
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import GSForge as gsf
import scipy

import holoviews as hv
hv.extension("bokeh")

***Declare paths used***

In [ ]:
# OS-independent path management.
from os import fspath, environ
from pathlib import Path

In [ ]:
OSF_PATH = Path(environ.get("GSFORGE_DEMO_DATA", default="~/GSForge_demo_data/osfstorage")).expanduser()
HYDRO_GEM_PATH = OSF_PATH.joinpath("AnnotatedGEMs", "oryza_sativa_hydro_raw.nc")
LIT_DGE_GSC_PATH = OSF_PATH.joinpath("Collections", "literature", "DGE")
LIT_TF_PATH = OSF_PATH.joinpath("Collections", "literature", "TF")
BORUTA_GSC_PATH = OSF_PATH.joinpath("Collections", "boruta")
assert HYDRO_GEM_PATH.exists()

***Load an AnnotatedGEM***

In [ ]:
agem = gsf.AnnotatedGEM(HYDRO_GEM_PATH)
agem

***Load GeneSetCollections***

In [ ]:
%%time
# lit_dge_coll = gsf.GeneSetCollection.from_folder(gem=agem, target_dir=LIT_DGE_GSC_PATH, name="Literature DGE")
# lit_tf_coll = gsf.GeneSetCollection.from_folder(gem=agem, target_dir=LIT_TF_PATH, name="Literature TF")
boruta_gsc = gsf.GeneSetCollection.from_folder(gem=agem, target_dir=BORUTA_GSC_PATH, name="Boruta Results")
# tf_geneset = gsf.GeneSet.from_GeneSets(*list(lit_tf_coll.gene_sets.values()), name='transcription factors')
# combined_gsc = gsf.GeneSetCollection(gem=agem, gene_sets={**boruta_gsc.gene_sets, 
#                                                           **lit_dge_coll.gene_sets,
#                                                           'transcription factors': tf_geneset})

In [ ]:
# combined_gsc

---

## Scoring and Judging GeneSets

In [ ]:
boruta_gsc["Boruta_treatment"]

In [ ]:
gene_rank_mdl = RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=-2)

***By feature importance***

In [ ]:
%%time
treatment_feature_importance = gsf.operations.RankGenesByModel(
    boruta_gsc,
    selected_gene_sets=["Boruta_treatment"],
    annotation_variables=["treatment"],
    model=gene_rank_mdl,
    n_iterations=5
)

**nFDR**

In [ ]:
%%time
treatment_nFDR = gsf.operations.nFDR(
    boruta_gsc,
    selected_gene_sets=["Boruta_treatment"],
    annotation_variables=["treatment"],
    model=gene_rank_mdl,
    n_iterations=5
)

**mProbes**

In [ ]:
%%time
treatment_mProbes = gsf.operations.analytics.mProbes(
    boruta_gsc,
    selected_gene_sets=["Boruta_treatment"],
    annotation_variables=["treatment"],
    model=gene_rank_mdl,
    n_iterations=5
)

Then you can optionally store such results to a GeneSet using `xarray.DataSet.update()`.

In [ ]:
boruta_gsc["Boruta_treatment"].data.update(treatment_feature_importance)
boruta_gsc["Boruta_treatment"].data.update(treatment_nFDR)
boruta_gsc["Boruta_treatment"].data.update(treatment_mProbes)
# boruta_gsc["Boruta_treatment"].data 

***Subset by supported genes***

In [ ]:
ds = boruta_gsc["Boruta_treatment"].data.sel(Gene=boruta_gsc["Boruta_treatment"].gene_support())
ds

In [ ]:
# class GeneSetVariableScatter():
    # gene_set, x_var, y_var, vdims=None

In [ ]:
def scatter_variables(dataset, x_var, y_var):
    """
    Directly plotting two variables (non-coordinates) against each other is not allowed from the holoviews.Dataset.to() interface.
    """
    df = pd.DataFrame({
        x_var: dataset[x_var].values,
        y_var: dataset[y_var].values,
        'Gene': dataset['Gene'].values
    })
    scatter = hv.Scatter(
        data=df,
        kdims=[x_var], 
        vdims=[y_var, 'Gene']
    )
    return scatter


opts = hv.opts.Scatter(xlabel='Feature Importance Score', ylabel='Estimated FDR', logx=True, size=1.5, height=300, width=300, bgcolor='lightgray', show_grid=True)

plot = scatter_variables(ds, 'feature_importance_mean', 'nFDR_mean').opts(opts)
plot

hv.save(plot, filename='figures/feature_importance_vs_FDR.png', fmt='png')

In [ ]:
plot

## Receiver Operating Characteristic (ROC)

Another way to evaluate a classification model is the [ROC](https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html#sphx-glr-auto-examples-model-selection-plot-roc-py).

Viewing ROC curves for multi-label models is a bit indirect as we have to use a binary classifier for each unique target label, so we provide this walkthrough.
Also examine the [sklearn demo](https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html).

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score, plot_roc_curve
from sklearn import preprocessing, model_selection

Get the count and annotation data from GSForge.

In [ ]:
counts, treatment = gsf.get_gem_data(boruta_gsc, annotation_variables=["treatment"], selected_gene_sets=["Boruta_treatment"])
classes = treatment.to_series().unique()
classes

Encode the annotation labels with a [one hot encoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder).

In [ ]:
enc = preprocessing.OneHotEncoder().fit(treatment.values.reshape(-1, 1))
treatment_onehot = enc.transform(treatment.values.reshape(-1, 1)).toarray()
treatment_onehot

Split the data and encoded annotations into a train and test set.

In [ ]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(counts, treatment_onehot)

Fit the model with the training data.

In [ ]:
%%time
roc_rf_model = OneVsRestClassifier(RandomForestClassifier(class_weight='balanced', max_depth=3,
                                   n_estimators=1000, n_jobs=-1))
roc_rf_model = roc_rf_model.fit(x_train, y_train)

Now predict class probabilities for the test count data.

In [ ]:
y_score = roc_rf_model.predict_proba(x_test)
y_score[:5]

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
for i, class_ in enumerate(classes):
    fpr[class_], tpr[class_], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[class_] = auc(fpr[class_], tpr[class_])

In [ ]:
roc_curves = {class_: hv.Curve((fpr[class_], tpr[class_]), ['False positive rate','True positive rate'])
              for class_ in classes}

In [ ]:
plot = hv.NdOverlay(roc_curves, kdims=['Treatment']).opts(padding=0.05, legend_position="bottom_right", width=300, height=300, show_grid=True, bgcolor='lightgray')
hv.save(plot, 'figures/ROC.png', 'png')
plot

## View a Decision Tree

In [ ]:
from sklearn import tree
import graphviz

Extract a single tree from a list of estimators.

In [ ]:
control_rfc = roc_rf_model.estimators_[0]
print(f"There are {len(control_rfc.estimators_)} trees.")

In [ ]:
selected_tree = control_rfc.estimators_[0]
graph = graphviz.Source(tree.export_graphviz(
    selected_tree, 
    feature_names=x_train.Gene.values,  
    class_names=["Not CONTROL", "CONTROL"],
    filled=True, 
    rounded=True,  
    special_characters=True))

graph
# graph.render('decision_tree', format="svg")  